# Guardian Scraper

## Dependencies

In [ ]:
import requests
import json
import time
import random

import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime, timedelta

import sys
from pathlib import Path

# Automatically detect the repo root (parent of notebook folder)
repo_root = Path().resolve().parent  # if notebook is in 'notebooks/' folder
sys.path.append(str(repo_root))

from config.config import get_environment

from config.config import data_import_json, data_export_json, data_import_pandas, data_export_pandas

## ENV

In [ ]:
ENV = get_environment(
    env_path="../environments",
    env_name="env.json"
)

# content_date = datetime.now().date() + timedelta(days=0)
content_date = ENV['CONTENT_DATE']
website = ENV['TARGET']["1"]['NAME']
version = ENV['VERSION']

url_scrape = ENV['TARGET']["1"]['URL']

reparse_only = ENV['SCRAPER']['REPARSE_ONLY']
continue_scraper = ENV['SCRAPER']['CONTINUE_SCRAPER']

## Mining

In [ ]:
# Generate looping count
range_limit = 100 # default maximum limit per page

headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,id;q=0.8',
    'authorization': '',
    'content-type': 'application/json',
    'newrelic': 'eyJ2IjpbMCwxXSwiZCI6eyJ0eSI6IkJyb3dzZXIiLCJhYyI6IjMyOTYzMjAiLCJhcCI6IjExMTk5OTg1NDciLCJpZCI6ImQ0Y2M2Nzk1M2RmMjc3MzUiLCJ0ciI6IjA3NmE1MDBiNjYwNzNkMDUzMGZhNTEwZjM2YmYxN2E2IiwidGkiOjE3NjQxNDkzODY3NzcsInRrIjoiMTMyMjg0MCJ9fQ==',
    'priority': 'u=1, i',
    'referer': 'https://www.guardianindonesia.co.id/skincare.html?page=1',
    'sec-ch-ua': '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'store': 'default',
    'traceparent': '00-076a500b66073d0530fa510f36bf17a6-d4cc67953df27735-01',
    'tracestate': '1322840@nr=0-1-3296320-1119998547-d4cc67953df27735----1764149386777',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36',
    'x-magento-cache-id': '2c3b02c55fa16a088ac257d464cf3c6604a62bf574d62984f52d6fda5f18ffdf',
    # 'cookie': 'private_content_version=69a83c7730183ab62c52e2c9b77d36dc; _gcl_au=1.1.487326357.1764085693; _tt_enable_cookie=1; _ttp=01KAXV6YEKK6S547A0KDCNHW3G_.tt.2; _gid=GA1.3.1844534993.1764085695; _fbp=fb.2.1764085698203.780253945897702889; _dc_gtm_UA-143423482-1=1; _gat_UA-143423482-1=1; ttcsid=1764149127881::odxZxED0C69bgue3KtGJ.2.1764149386381.0; ttcsid_CTHA7FJC77UD328UMEL0=1764149127880::eUyh6q8NcBBiW7vk3DDm.2.1764149386381.0; _ga=GA1.1.275656929.1764085694; _ga_6KEZ58MRZ7=GS2.1.s1764148446$o2$g1$t1764149386$j58$l0$h0',
}

total_pages = 0
loop_count = 0
url_scrape = 'https://www.guardianindonesia.co.id/graphql'
while loop_count < total_pages or loop_count == 0:
    page = loop_count+1
    params = {
        # 'query': 'query GetCategories($id:String!$pageSize:Int!$currentPage:Int!$filters:ProductAttributeFilterInput!$sort:ProductAttributeSortInput){categories(filters:{category_uid:{in:[$id]}}){items{uid ...CategoryFragment __typename}__typename}products(pageSize:$pageSize currentPage:$currentPage filter:$filters sort:$sort){...ProductsFragment __typename}}fragment CategoryFragment on CategoryTree{id uid meta_title meta_keywords meta_description image __typename}fragment ProductsFragment on Products{items{id uid categories{id uid name level __typename}name price_range{minimum_price{discount{amount_off percent_off __typename}final_price{currency value __typename}regular_price{currency value __typename}__typename}maximum_price{regular_price{currency value __typename}final_price{currency value __typename}discount{amount_off percent_off __typename}__typename}__typename}sku special_price price{regularPrice{amount{currency value __typename}__typename}__typename}small_image{url __typename}stock_status stock_item{qty min_qty min_sale_qty max_sale_qty backorders qty_increments manage_stock is_in_stock __typename}type_id url_key url_suffix sale promo __typename}page_info{total_pages current_page page_size __typename}total_count __typename}',
        'query': 'query GetCategoriesFull($id:String!$pageSize:Int!$currentPage:Int!$filters:ProductAttributeFilterInput!$sort:ProductAttributeSortInput){categories(filters:{category_uid:{in:[$id]}}){items{uid ...CategoryFragment __typename children{uid id name meta_title meta_keywords meta_description image __typename}}__typename}products(pageSize:$pageSize currentPage:$currentPage filter:$filters sort:$sort){items{id uid type_id url_key name sku sale promo special_price stock_status small_image{url __typename}price{regularPrice{amount{currency value __typename}__typename}__typename}price_range{minimum_price{discount{amount_off percent_off __typename}final_price{currency value __typename}regular_price{currency value __typename}__typename}maximum_price{discount{amount_off percent_off __typename}final_price{currency value __typename}regular_price{currency value __typename}__typename}__typename}stock_item{qty min_qty min_sale_qty max_sale_qty is_in_stock __typename}how_to_use description{html __typename}meta_description more_info{label value __typename}review{reviews_count rating_summary __typename}media_gallery_entries{id label position disabled file __typename}__typename}page_info{total_pages current_page page_size __typename}total_count __typename}}fragment CategoryFragment on CategoryTree{id uid name meta_title meta_keywords meta_description image __typename}',
        'operationName': 'GetCategories',
        'variables': f'{{"currentPage":{page},"id":"MjUz","filters":{{"category_uid":{{"eq":"MjUz"}}}},"pageSize":{range_limit},"sort":{{"position":"ASC","stock_status":"DESC"}}}}',
    }

    print(f"Get Items Loop Count: {page}/{total_pages} | Range Items: {range_limit} | URL: {url_scrape}")

    response = requests.get(
        url_scrape,
        params=params,
        headers=headers
    )

    print(f"Response Items Loop Count: {page}/{total_pages} | Range Items: {range_limit} | URL: {url_scrape} | STATUS CODE: {response.status_code}")

    try:
        total_pages = response.json()['data']['products']['page_info']['total_pages']

        data_export_json(
            data=response.json(),
            website=website,
            folder_name=f'scraper/{website}',
            version=version,
            content_date=content_date, # "0000-00-00"
            additional_info=f"scrape-page{page}",
            metadata={
                "status_code": response.status_code,
                "page": page,
                "range_limit": range_limit,
                "url_scrape": url_scrape
            }
        )

    except Exception as e:
        data_export_json(
            data=response.text,
            website=website,
            folder_name=f'scraper/{website}',
            version=version,
            content_date=content_date, # "0000-00-00"
            additional_info=f"scrape-page{page}",
            metadata={
                "status_code": response.status_code,
                "error": e,
                "page": page,
                "range_limit": range_limit,
                "url_scrape": url_scrape
            }
        )

    loop_count += 1
    if total_pages == 0:
        break

    time.sleep(0.5)

## Parser

In [ ]:
def parser_item(
        response_json: dict,
        page: int,
        scrape_date: datetime
    ):

    parsed_list = list()
    len_items = len(response_json['data']['products']['items'])
    cat_name = response_json['data']['categories']['items'][0]['name']
    for index, item in enumerate(response_json['data']['products']['items']):

        try:
            item_id = item['id']
            item_name = item['name']
            item_slug = item['url_key']
            item_brand = [v for v in item['more_info'] if v['label'] == 'Brand'][0]['value']
            stock = item['stock_item']['qty']
            item_url = f"https://guardianindonesia.co.id/{item_slug}.html"
            url_image = item['small_image']['url']
            review_total = item['review']['reviews_count']
            review_rating = item['review']['rating_summary']

            data_dict = dict()
            data_dict['scrape_date'] = str(scrape_date)
            data_dict['category_slug'] = None
            data_dict['category'] = cat_name
            data_dict['slug'] = item_slug
            data_dict['id'] = item_id
            data_dict['brand'] = item_brand
            data_dict['name'] = item_name
            data_dict['url'] = item_url
            data_dict['url_image'] = url_image
            data_dict['review_total'] = review_total
            data_dict['review_rating'] = review_rating
            data_dict['review_recommended'] = None
            data_dict['wishlist'] = None
            data_dict['variant_id'] = None
            data_dict['variant_name'] = None
            data_dict['stock'] = stock
            data_dict['ean'] = None
            data_dict['currency'] = 'IDR'
            data_dict['price_range'] = f"{item['price_range']['minimum_price']['final_price']['value']} - {item['price_range']['maximum_price']['final_price']['value']}"
            data_dict['price'] = item['price_range']['maximum_price']['regular_price']['value']
            data_dict['price_after_disc'] = item['price_range']['maximum_price']['final_price']['value']
            data_dict['price_disc_type'] = None
            data_dict['price_disc'] = item['price_range']['maximum_price']['discount']['amount_off']
            data_dict['price_disc_perc'] = item['price_range']['maximum_price']['discount']['percent_off']
            data_dict['is_package'] = None

        except Exception as e:
            data_dict = dict()
            print(f"ERROR Parser Page: {page} | Item Index: {index} | Item Count: {index+1}/{len_items} | URL: {item_url} | {e}")

        parsed_list.append(data_dict)

    return parsed_list

In [ ]:
# Generate looping count
range_limit = 100 # default maximum limit per page

total_pages = 0
loop_count = 0
url_scrape = 'https://www.guardianindonesia.co.id/graphql'
parsed_list = list()
try:
    while loop_count < total_pages or loop_count == 0:
        page = loop_count+1

        print(f"Parsing Items Loop Count: {page}/{total_pages} | Range Items: {range_limit} | URL: {url_scrape}")

        try:
            response_json = data_import_json(
                website=website,
                folder_name=f'scraper/{website}',
                version=version,
                content_date=content_date, # "0000-00-00"
                additional_info=f"scrape-page{page}"
            )

            response_json = response_json['data']

            parsed_list_temp = parser_item(
                response_json=response_json,
                page=page,
                scrape_date=content_date
            )

            total_pages = response_json['data']['products']['page_info']['total_pages']

        except Exception as e:
            parsed_list_temp = list()
            print(e)
            break

        parsed_list = parsed_list + parsed_list_temp

        loop_count += 1
        if total_pages == 0:
            break


    # Convert all list to DataFrame
    df_parse = pd.DataFrame(parsed_list)

    data_export_pandas(
        df_output=df_parse,
        website=website,
        folder_name=f'parser/{website}',
        version=version,
        content_date=content_date, # "0000-00-00"
        additional_info="parsed",
        incl_excel=True
    )

except Exception as e:
    print(e)

    df_parse = pd.DataFrame(parsed_list)
    data_export_pandas(
        df_output=df_parse,
        website=website,
        folder_name=f'parser/{website}',
        version=version,
        content_date=content_date, # "0000-00-00"
        additional_info="parsed",
        incl_excel=True
    )